In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from pandas_datareader import data as pdr
import yfinance as yf


In [ ]:
stocklist = ['CBA', 'BHP', 'TLS', 'NAB', 'WBC', 'STO']
mc_sims = 1000
T=100 #timeframe in days
initial_portfolio = 1000

count = 1000
def random_port_size():
  def get_data(stocks, start, end):
      data = yf.download(stocks, start, end)["Close"]
      returns = data.pct_change()
      meanReturn = returns.mean()
      covMatrix = returns.cov()
      return meanReturn, covMatrix

  stocks = [stock + ".AX" for stock in stocklist]
  endDate = dt.datetime.now()
  startDate = endDate - dt.timedelta(days=300)
  meanReturns, covMatrix = get_data(stocks, startDate, endDate)
  weights = np.random.random(len(meanReturns))
  #ensure that the sum of the weights array is '1'
  weights /= np.sum(weights)
  meanM = np.full(shape=(T, len(weights)), fill_value=meanReturns)
  meanM = meanM.T
  portfolio_sims = np.full(shape=(T, mc_sims), fill_value=0.0)

  for m in range(0, mc_sims):
    Z = np.random.normal(size=(T, len(weights)))
    L = np.linalg.cholesky(covMatrix)
    daily_returns = meanM + np.inner(L, Z)
    portfolio_sims[:, m] = np.cumprod(np.inner(weights, daily_returns.T)+1)*initial_portfolio

  """
  plt.plot(portfolio_sims)
  plt.ylabel('Portfolio Value($)')
  plt.xlabel('Days')
  plt.title('MC simulation of a stock portfolio')
  plt.show()
  """

  # Calculate the final portfolio values for each simulation
  final_portfolio_values = portfolio_sims[-1]

  # Calculate the percentage change from the initial portfolio value
  returns = (final_portfolio_values - initial_portfolio) / initial_portfolio

  def num_calculator(value_in_pct):
      # Count the number of simulations with at least a specified increase
      num_simulations = np.sum(returns >= (value_in_pct / 100)) / len(returns) * 100
      return num_simulations

  # Reshape weights and create a DataFrame
  new_weights = weights.reshape(1, weights.shape[0])
  weights_df = pd.DataFrame(new_weights, index=['Weights'], columns=stocklist)
  print('\n')
  print(weights_df)
  print('\n')
  # Print the result
  print("Percentage of simulations with at least no loss:", num_calculator(0))
  print("Percentage of simulations with at least a 10% increase:", num_calculator(10))
  print("Percentage of simulations with at least a 20% increase:", num_calculator(20))
  print("Percentage of simulations with at least a 30% increase:", num_calculator(30))
  print("Percentage of simulations with at least a 40% increase:", num_calculator(40))
  print("Percentage of simulations with at least a 50% increase:", num_calculator(50))

  return weights_df, num_calculator(10), num_calculator(20), num_calculator(30), num_calculator(40), num_calculator(50)

In [ ]:
best_10 = 0
best_weights_10 = None
best_20 = 0
best_weights_20 = None
best_30 = 0
best_weights_30 = None
best_40 = 0
best_weights_40 = None
best_50 = 0
best_weights_50 = None

for i in range(count):
  weighted_df, returns_10, returns_20, returns_30, returns_40, returns_50 = random_port_size()
  if returns_10 > best_10:
    best_10 = returns_10
    best_weights_10 = weighted_df
  if returns_20 > best_20:
    best_20 = returns_20
    best_weights_20 = weighted_df
  if returns_30 > best_30:
    best_30 = returns_30
    best_weights_30 = weighted_df
  if returns_40 > best_40:
    best_40 = returns_40
    best_weights_40 = weighted_df
  if returns_50 > best_50:
    best_50 = returns_50
    best_weights_50 = weighted_df



[*********************100%%**********************]  6 of 6 completed
[                       0%%                      ]



              CBA       BHP       TLS       NAB       WBC       STO
Weights  0.319038  0.184744  0.030111  0.092207  0.133045  0.240855


Percentage of simulations with at least no loss: 64.8
Percentage of simulations with at least a 10% increase: 21.099999999999998
Percentage of simulations with at least a 20% increase: 2.1
Percentage of simulations with at least a 30% increase: 0.1
Percentage of simulations with at least a 40% increase: 0.0
Percentage of simulations with at least a 50% increase: 0.0


[*********************100%%**********************]  6 of 6 completed
[*********************100%%**********************]  6 of 6 completed



              CBA       BHP       TLS       NAB      WBC       STO
Weights  0.242485  0.225456  0.012018  0.209495  0.18719  0.123355


Percentage of simulations with at least no loss: 59.5
Percentage of simulations with at least a 10% increase: 13.4
Percentage of simulations with at least a 20% increase: 1.5
Percentage of simulations with at least a 30% increase: 0.1
Percentage of simulations with at least a 40% increase: 0.0
Percentage of simulations with at least a 50% increase: 0.0



[                       0%%                      ]

...



              CBA       BHP       TLS       NAB       WBC       STO
Weights  0.148155  0.026273  0.182685  0.236108  0.173164  0.233615


Percentage of simulations with at least no loss: 63.0
Percentage of simulations with at least a 10% increase: 16.7
Percentage of simulations with at least a 20% increase: 2.1
Percentage of simulations with at least a 30% increase: 0.1
Percentage of simulations with at least a 40% increase: 0.1
Percentage of simulations with at least a 50% increase: 0.0


[                       0%%                      ]



              CBA       BHP       TLS       NAB       WBC       STO
Weights  0.169161  0.176627  0.050081  0.256909  0.136005  0.211218


Percentage of simulations with at least no loss: 65.8
Percentage of simulations with at least a 10% increase: 17.4
Percentage of simulations with at least a 20% increase: 1.0999999999999999
Percentage of simulations with at least a 30% increase: 0.0
Percentage of simulations with at least a 40% increase: 0.0
Percentage of simulations with at least a 50% increase: 0.0


[*********************100%%**********************]  6 of 6 completed




              CBA       BHP       TLS       NAB       WBC       STO
Weights  0.221435  0.161489  0.156483  0.041963  0.238067  0.180563


Percentage of simulations with at least no loss: 67.7
Percentage of simulations with at least a 10% increase: 18.4
Percentage of simulations with at least a 20% increase: 2.5
Percentage of simulations with at least a 30% increase: 0.0
Percentage of simulations with at least a 40% increase: 0.0
Percentage of simulations with at least a 50% increase: 0.0


In [ ]:
print('> 10%', best_10)
print(best_weights_10)
print('\n')
print('> 20%', best_20)
print(best_weights_20)
print('\n')
print('> 30%', best_30)
print(best_weights_30)
print('\n')
print('> 40%', best_40)
print(best_weights_40)
print('\n')
print('> 50%', best_50)
print(best_weights_50)


> 10% 48.4
              CBA       BHP       TLS       NAB       WBC       STO
Weights  0.036073  0.190184  0.556125  0.020573  0.047059  0.149985


> 20% 15.7
              CBA       BHP       TLS       NAB       WBC       STO
Weights  0.042185  0.324663  0.370015  0.025386  0.027506  0.210244


> 30% 2.8000000000000003
              CBA       BHP       TLS       NAB       WBC       STO
Weights  0.147179  0.199298  0.334058  0.012872  0.007995  0.298597


> 40% 0.5
              CBA       BHP       TLS       NAB       WBC       STO
Weights  0.042185  0.324663  0.370015  0.025386  0.027506  0.210244


> 50% 0
None



**include value at risk(vAr) and conditional value at risk(CVAR)**

In [ ]:
def mcVar(returns, alpha = 5):
  """
  Input: panda series of returns
  output: percentile on return distribution based on confidence level
  """

  if isinstance(returns, pd.Series):
    return np.percentile(returns, alpha)

  else:
    raise TypeError("Input must be a pandas series")


def mcCVar(returns, alpha = 5):
  """
  Input: panda series of returns
  output: percentile on return distribution based on confidence level
  """

  if isinstance(returns, pd.Series):
    belowVar = returns<= mcVar(returns, alpha = alpha)
    return returns[belowVar].mean()

  else:
    raise TypeError("Input must be a pandas series")


portResult = pd.Series(portfolio_sims[-1, :])

VaR = initial_portfolio - mcVar(portResult, alpha=5)
CVar = initial_portfolio - mcCVar(portResult, alpha=5)


print(f'Value at risk: ${VaR}')
print(f'Conditional value at risk: ${CVar}')
